# Tutorial: Hydrogen Electro-Oxidation Reaction

## Introduction
 The microkinetic model of hydrogen electro-oxidation (**HOR**) proposed by Wang et al. [1]. is implemented here. The HOR global reaction is :
$$ \ce{H2 + Pt <--> 2 H+ + 2e-}  $$
The reaction mechanism is described by three elementary reaction steps
$$
\begin{align}
&\ce{H2 + 2 Pt <--> 2 H^{*}} & \text{dissociative adsortion} \tag{DA} \\
&\ce{H2 + Pt <--> H^{*} + H+ + e-} & \text{oxidative adsorption} \tag{OA} \\
&\ce{H^{*} <--> Pt + H+ + e-} & \text{oxidative desorption} \tag{OD}
\end{align}
$$

Wang et al. [1] analyzed the adsorption isotherms of H on adatom (Atop) sites and hollow/bridge (H/B) sites.

- **Atop sites**: These are positions where the adsorbed atom is located directly on a substrate atom (i.e., adsorption at a vertex position).
- **Hollow/Bridge (H/B) sites**:
  - *Hollow sites*: Located at the center of several substrate atoms, such as 3- or 4-coordinate sites in a crystal cell.
  - *Bridge sites*: Intermediate positions between two substrate atoms.


## Input files
The corresponding input files are available in the software repository's tutorials/example folder.

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from melektrodica import Collector, Calculator, Coordinator, Fitter, Kpynetic, Writer

In [8]:
directory = os.path.join("examples","Wang2007Hydrogen")
writer = Writer(log_file="Wang2007Hydrogen.log", log_directory=directory)
data = Collector(directory)
Kpy = Kpynetic(data)

INFO: ***  Collector  ***
INFO: Reading parameters from file: examples/Wang2007Hydrogen/parameters.md
INFO: Parameters:
INFO: Anode: True
INFO: Operation conditions:
		Temperature: 296.15 K
		Overpotential: [0.0: 0.001: 0.5] V
INFO: Pre-exponential factor: 500.0 (from j*)
INFO: Thermochemical part details:
	DG_reaction: False
	G_formation: True
INFO: Reading Species data from file: examples/Wang2007Hydrogen/species.md
INFO: Species lists:
	Reactants: ['H2']
	Products: ['H+']
	Adsorbed species: ['H*']
	Catalysts: ['Pt']
INFO: Initial concentrations processed.
INFO: Catalyst matrix created.
INFO: Formation energies processed
INFO: Reading Reactions data from file: examples/Wang2007Hydrogen/reactions.md
INFO: Reaction matrix processed.
INFO: Thermochemical reactions parameters processed.
INFO: ***  Data collection completed successfully.  ***

INFO: *** Kpynetic :  ***


## References
1. Wang, J. X., Springer, T. E., Liu, P., Shao, M., and Adzic, R. R. (2007). Hydrogen oxidation reaction on Pt in acidic media: Adsorption isotherm and activation free energies. The Journal of Physical Chemistry C, 111(33), 12425-12433, https://doi.org/10.1021/jp073400i.